## AAPL Stock Price Prediction by Rolling ARIMA Model on Spark
#### Author: ixihao
#### Environment: Databricks
#### Language: Python (pyspark) & Scala
#### Package: sparkts_0.4.1
#### Data: Stock price of Apple from Yahoo finance.
#### Description: In this project, I try to build a ARIMA model to predict the AAPL stock price and evaluate the predict result. First, I will load and preprocess the data download from Yahoo Finance. Then, initialize the ARIMA model in scala.

In [2]:
#import math
#import matplotlib
#import numpy as np
import pandas as pd

#import seaborn as sns
#import time
#import statsmodels.api as sm

#from pyspark.sql.functions import isnull
#from pyspark.sql import functions as F
#from datetime import date, datetime, time, timedelta
from pyspark.sql.types import DateType
from pyspark.sql.functions import lit

# File location and type
file_location = "/FileStore/tables/AAPL__1_-bf1ee.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
aapl_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(aapl_df.take(5))

Date,Open,High,Low,Close,Adj Close,Volume
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.023007,117258400
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.021807,43971200
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.020206,26432000
1980-12-17,0.462054,0.464286,0.462054,0.462054,0.020706,21610400
1980-12-18,0.475446,0.477679,0.475446,0.475446,0.021307,18362400


In [3]:
# Check the datatype of each column
aapl_df.dtypes

Out[ 2 ]: 
[('Date', 'string'),
 ('Open', 'string'),
 ('High', 'string'),
 ('Low', 'string'),
 ('Close', 'string'),
 ('Adj Close', 'string'),
 ('Volume', 'string')]

In [4]:
# Transform columns to the appropriate datatype and remove columns we don't need
price_df = aapl_df.select(aapl_df.Date.cast(DateType()).alias('Date'),
               aapl_df.Close.cast('float').alias('Close'))

# Check the datatype of each column
price_df.dtypes

Out[ 5 ]: [('Date', 'date'), ('Close', 'float')]

In [5]:
# Remove rows that contain Null value (1 Null record)
price_df = price_df.filter("Date is not null and Close is not null")

# Check the size of dataframe
price_df.count()

Out[ 6 ]: 9594

In [6]:
# Add a column 'Stock' which is a string key of the stock price in order to prepare for the TimeSeriesRDD in Spark_TimeSeries library
price_df = price_df.withColumn("Stock",lit("AAPL"))

# Store the dataframe into a csv file in order to share with Scala REPL
price_df.write.option("header", "true").csv("/FileStore/tables/aapl_price4.csv")

display(price_df.take(5))

Date,Close,Stock
1980-12-12,0.513392984867096,AAPL
1980-12-15,0.48660698533058167,AAPL
1980-12-16,0.45089301466941833,AAPL
1980-12-17,0.4620540142059326,AAPL
1980-12-18,0.4754459857940674,AAPL


In [7]:
%scala
import org.apache.spark.sql.functions.to_timestamp
import org.apache.spark.sql.types.DoubleType
import com.cloudera.sparkts.models.{ARIMA, ARIMAModel}
import com.cloudera.sparkts.{DateTimeIndex, DayFrequency, TimeSeriesRDD}
import java.time.{ZoneId, ZonedDateTime}
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.linalg.{Vectors, Vector}
import org.apache.spark.mllib.linalg.DenseVector
import org.joda.time.DateTime

// Load data from file in HDFS and format the data
val path = "/FileStore/tables/aapl_price4.csv"
val formattedData = spark.read.option("header", "true").csv(path)
val priceDf = formattedData
      .withColumn("Date", to_timestamp(formattedData("date"), "yyyy-MM-dd"))
      .withColumn("Close", formattedData("close").cast(DoubleType))
      .sort("Date")

//val dd = priceDf.filter("Date > '2010-10-22 00:00:00'")
//val dd = priceDf.filter((unix_timestamp(current_timestamp()) - $"timestamp")/60 < 10)
/*
val minDate = priceDf.select(min("Date")).collect()(0).getTimestamp(0)
val maxDate = priceDf.select(max("Date")).collect()(0).getTimestamp(0)
val zone = ZoneId.systemDefault()
val dtIndex = DateTimeIndex.uniformFromInterval(
      ZonedDateTime.of(minDate.toLocalDateTime, zone),
      ZonedDateTime.of(maxDate.toLocalDateTime, zone),
      new DayFrequency(1))
*/

// Extract date from priceDF
val dateList = priceDf.select("Date").collect()
val dateListLength = dateList.size

// Initialize Array[Double] to store the predicted price
var day: Array[Double] = Array()
var week1: Array[Double] = Array()
var week2 : Array[Double] = Array()
var month1: Array[Double] = Array()
var month4 : Array[Double] = Array()


val rollingNum = 2000
val trainingNum = 100
// Moving ARIMA start
for(i <- dateListLength-rollingNum to dateListLength-1){
  println("### No." + i)
  // Initialize a TimeSeriesRDD
  val minDate = dateList(i-trainingNum).getTimestamp(0).toLocalDateTime()
  val maxDate = dateList(i).getTimestamp(0).toLocalDateTime()
  val zone = ZoneId.systemDefault()
  val dtIndex = DateTimeIndex.uniformFromInterval(
        ZonedDateTime.of(minDate, zone),
        ZonedDateTime.of(maxDate, zone),
        new DayFrequency(1))
  val tsRdd: TimeSeriesRDD[String] = TimeSeriesRDD.timeSeriesRDDFromObservations(
        targetIndex = dtIndex,
        df = priceDf,
        tsCol = "Date",
        keyCol = "Stock",
        valueCol = "Close")
  // Number of days we want to predict (26 Trading days a month, 4 months)
  val days = 104
  println(minDate)
  println(maxDate)
  // Fill in missing values based on linear interpolation (Not everyday is trading day)
  val tsRdd_filled = tsRdd.fill("linear")
  
  // Find the best parameter in ARIMA
  val arimaAndVectorRdd = tsRdd_filled.map {
        case (key, denseVector) =>
          (ARIMA.autoFit(denseVector), denseVector)
      }
  val coefficients = arimaAndVectorRdd.map {
        case (arimaModel, denseVector) => {
          (arimaModel.coefficients.mkString(","),
            (arimaModel.p, arimaModel.d, arimaModel.q))
        }
      }
  coefficients.collect().foreach {
        case (coefficients, (p, d, q)) =>
          println("coefficients:" + coefficients + "=>" + "p = " + p + ", d = " + d + ", q = " + q)
      }

  // Predict next n days value
  val forecast = arimaAndVectorRdd.map {
    case (arimaModel, denseVector) => {
      arimaModel.forecast(denseVector, days)
    }
  }
  val forecastValue = forecast.map(_.toArray.mkString(","))

  // Extrace predicted value (Remove the past value)
  val preditcedValueRdd = forecastValue.map { parts =>
    val partArray = parts.split(",")
    for (i <- partArray.length - days until partArray.length) yield partArray(i)
  }.map(_.toArray.mkString(","))
  
  /*
  // Print the predicted value
  preditcedValueRdd.collect().foreach { row =>
    println("forecast of next " + days + " observations: " + row)
  }
  */
  // Update the Array[Double] initialized before
  //val forvalue: Array[Double] = preditcedValueRdd.toArray()
  val forvalue: Array[Double] = preditcedValueRdd.collect().mkString("").split(",").map(_.toDouble)
  //val forvalue: Array[Double] = preditcedValueRdd.collect().size
  // 5 trading days a week, 26 trading days a month
  day :+= forvalue(0)
  week1 :+= forvalue(4) 
  week2 :+= forvalue(9)
  month1 :+= forvalue(25) 
  month4 :+= forvalue(103)
  println(forvalue(0))
  println(forvalue(4))
  println(forvalue(9))
  println(forvalue(25))
  println(forvalue(103))
}
/*
//Version 2

//val features = priceDf.select("features")
//val featurestd = features.rdd.map(_.getAs[Vector]("features"))
//val tt = priceDf.head(2000).select("Close").as[Double].collect()
val stock_array = priceDf.select("Close").as[Double].collect()
val stock_array_length = stock_array.size

var day = Array(0.0)
var week1 = Array(0.0)
var week2 = Array(0.0)
var month1 = Array(0.0)
var month4 = Array(0.0)

for(i <- 0 to stock_array_length){
println(i)
      
var train_data_array : Array[Double] = stock_array.slice(i,i+100)
val train_data_vector = new DenseVector(train_data_array)
//println(train_data_vector.size)

val maxP = 20
val maxD = 10
val maxQ = 20
val arima = ARIMA.autoFit(train_data_vector,maxP,maxD,maxQ)
println("is stationary:"+arima.isStationary())
println("AIC value:"+arima.approxAIC(train_data_vector))
print(s"best model：p is:${arima.p},d is:${arima.d},q is ${arima.q}")

//Train the values

val arimaModel = ARIMA.fitModel(arima.p, arima.d, arima.q, train_data_vector)


//Get the forecasted value and convert into the dense vector
val forecasted = arimaModel.forecast(train_data_vector, days)
val forvalue = new DenseVector(forecasted.toArray.slice(forecasted.size - days, forecasted.size))
//println(forvalue.size)
day :+= forvalue(0)
week1 :+= forvalue(4) 
week2 :+= forvalue(9)
month1 :+= forvalue(25) 
month4 :+= forvalue(103) 
}
*/
/*
val arimaModel = ARIMA.fitModel(1, 0, 1, featuresRDD)
println("coefficients: " + arimaModel.coefficients)
val forecast = arimaModel.forecast(ts, 20)
println("forecast of next 20 observations: " + forecast.toArray.mkString(","))
*/

### No.7594
2010-08-27T00:00
2011-01-20T00:00
coefficients:0.0887921641219848,0.04485432739488617=>p = 0, d = 1, q = 1
47.57171964302848
47.92688829951641
48.37084912012633
49.79152374607806
56.71731254759276
### No.7595
2010-08-30T00:00
2011-01-21T00:00
coefficients:1.1498260181434148,-0.6669835658563529,-0.009588858962733645,-1.1667647736810642,0.9046777594019735=>p = 3, d = 1, q = 2
46.15036956266029
45.31899354276778
45.55520289343614
45.57788294917334
45.57958402717618
### No.7596
2010-08-31T00:00
2011-01-24T00:00
coefficients:0.08299789351634455,0.02853214448903696=>p = 1, d = 1, q = 0
48.292921958338255
48.634674344545246
49.06185210985587
50.42882095882829
57.092794097568834
### No.7597
2010-09-01T00:00
2011-01-25T00:00
coefficients:0.004829593163118008,1.8477190061217674,-0.8229609300930728,-0.12838509027290015,0.04140051060013477,0.0036445109525447866,-1.8760824484231737,0.9110928732689368=>p = 5, d = 1, q = 2
51.223409130739384
68.66460363687466
87.13621097291399
63.01391302098814
78.85513766328803
### No.7598
2010-09-02T00:00
2011-01-26T00:00
coefficients:0.08132240853265235,0.025098186162007374=>p = 1, d = 1, q = 0
49.205148908554904
49.538820701955714
49.95590069595586
51.29055667674642
57.79700458310039
### No.7599
2010-09-03T00:00
2011-01-27T00:00
coefficients:0.08245029600760831,0.08096396636979682=>p = 0, d = 1, q = 1
49.096857069685576
49.426658253716006
49.83890973375404
51.15811446987576
57.589237558469144
### No.7600
2010-09-07T00:00
2011-01-28T00:00
coefficients:0.07158288718097555,0.01933373716337948=>p = 1, d = 1, q = 0
48.08721867597723
48.37919401081916
48.74416469398453
49.91207088011423
55.60561353749651
### No.7601
2010-09-08T00:00
2011-01-31T00:00
coefficients:0.0752810831096298,0.08740225010794919=>p = 0, d = 1, q = 1
48.55590657109014
48.85703090352866
49.23343631907681
50.437933648830885
56.30985813138201
### No.7602
2010-09-09T00:00
2011-02-01T00:00
coefficients:0.08116975773436388,0.08920707588753084=>p = 0, d = 1, q = 1
49.43616268229035
49.76084171322781
50.16669050189964
51.46540662564948
57.796647728929955
### No.7603
2010-09-10T00:00
2011-02-02T00:00
coefficients:0.07963156036370422,0.08271883030426162=>p = 0, d = 1, q = 1
49.248228839779685
49.56675508123449
49.964912883052996
51.239017848872216
57.45027955724091
### No.7604
2010-09-13T00:00
2011-02-03T00:00
coefficients:0.07625291212117856,0.08409175561942055=>p = 0, d = 1, q = 1
49.123821915465214
49.42883356394992
49.81009812455581
51.03014471849464
56.97787186394645
### No.7605
2010-09-14T00:00
2011-02-04T00:00
coefficients:0.12281306674833732,-0.6738139151744906,0.019148617507923037,0.795424160535957,-0.028671973505665093=>p = 2, d = 1, q = 2
49.43359733768894
49.77477558345376
50.16238042586881
51.347555861470696
57.13688723737948
### No.7606
2010-09-15T00:00
2011-02-07T00:00
coefficients:0.07262531806256328,0.02269523827756883=>p = 1, d = 1, q = 0
50.342975523728164
50.640225071395434
51.01178428663285
52.20077377539075
57.9970975330855
### No.7607
2010-09-16T00:00
2011-02-08T00:00
coefficients:0.07751875115698864,0.09179188191032804=>p = 0, d = 1, q = 1
50.8554216457819
51.16549665040985
51.553090406194784
52.79339042470657
58.839853014951544
### No.7608
2010-09-17T00:00
2011-02-09T00:00
coefficients:0.08197440074944115,0.09882784133360568=>p = 0, d = 1, q = 1
51.27769790851841
51.605595511516164
52.01546751526335
53.32705792725436
59.72106118571051
### No.7609
2010-09-20T00:00
2011-02-10T00:00
coefficients:0.07083698920229788,0.08762784024949671=>p = 0, d = 1, q = 1
50.66544190227365
50.948789859082844
51.30297480509434
52.436366632331115
57.96165179011041
### No.7610
2010-09-21T00:00
2011-02-11T00:00
coefficients:0.06824899064090251,0.01343369296212985=>p = 1, d = 1, q = 0
51.0476455006962
51.324357303233306
51.6702488573743
52.777101830625604
58.173010075225726
### No.7611
2010-09-22T00:00
2011-02-14T00:00
coefficients:0.07043398921454669,0.08021778486159789=>p = 0, d = 1, q = 1
51.38485963007737
51.66659558693557
52.0187655

In [8]:
%scala
case class Row(day: Double, week1: Double, week2: Double, month1: Double, month4: Double)

val xs = Array(day, week1, week2, month1, month4).transpose
val rdd = sc.parallelize(xs).map(ys => Row(ys(0), ys(1), ys(2), ys(3), ys(4)))
val dfdd = rdd.toDF("day", "week1", "week2", "month1", "month4")
dfdd.display()

+------------------+------------------+------------------+------------------+------------------+
 day| week1| week2| month1| month4|
+------------------+------------------+------------------+------------------+------------------+
 47.57171964302848| 47.92688829951641| 48.37084912012633| 49.79152374607806| 56.71731254759276|
 46.15036956266029| 45.31899354276778| 45.55520289343614| 45.57788294917334| 45.57958402717618|
48.292921958338255|48.634674344545246| 49.06185210985587| 50.42882095882829|57.092794097568834|
51.223409130739384| 68.66460363687466| 87.13621097291399| 63.01391302098814| 78.85513766328803|
49.205148908554904|49.538820701955714| 49.95590069595586| 51.29055667674642| 57.79700458310039|
49.096857069685576|49.426658253716006| 49.83890973375404| 51.15811446987576|57.589237558469144|
 48.08721867597723| 48.37919401081916| 48.74416469398453| 49.91207088011423| 55.60561353749651|
 48.55590657109014| 48.85703090352866| 49.23343631907681|50.437933648830885| 56.30985813138201|
 49.43616268229035| 49.76084171322781| 50.16669050189964| 51.46540662564948|57.796647728929955|
49.248228839779685| 49.56675508123449|49.964912883052996|51.239017848872216| 57.45027955724091|
49.123821915465214| 49.42883356394992| 49.81009812455581| 51.03014471849464| 56.97787186394645|
 49.43359733768894| 49.77477558345376| 50.16238042586881|51.347555861470696| 57.13688723737948|
50.342975523728164|50.640225071395434| 51.01178428663285| 52.20077377539075| 57.9970975330855|
 50.8554216457819| 51.16549665040985|51.553090406194784| 52.79339042470657|58.839853014951544|
 51.27769790851841|51.605595511516164| 52.01546751526335| 53.32705792725436| 59.72106118571051|
 50.66544190227365|50.948789859082844| 51.30297480509434|52.436366632331115| 57.96165179011041|
 51.0476455006962|51.324357303233306| 51.6702488573743|52.777101830625604|58.173010075225726|
 51.38485963007737| 51.66659558693557| 52.01876553300832| 53.14570936044112| 58.639560519176|
 51.48100452979667|51.747848460091944|52.081403294100554| 53.1487787629281|58.352234173462406|
 51.97727344080777| 52.25691762793891|52.606472861852836| 53.7250496103774| 59.17811125943463|
+------------------+------------------+------------------+------------------+------------------+
only showing top 20 rows

defined class Row
xs: Array[Array[Double]] = Array(Array(47.57171964302848, 47.92688829951641, 48.37084912012633, 49.79152374607806, 56.71731254759276), Array(46.15036956266029, 45.31899354276778, 45.55520289343614, 45.57788294917334, 45.57958402717618), Array(48.292921958338255, 48.634674344545246, 49.06185210985587, 50.42882095882829, 57.092794097568834), Array(51.223409130739384, 68.66460363687466, 87.13621097291399, 63.01391302098814, 78.85513766328803), Array(49.205148908554904, 49.538820701955714, 49.95590069595586, 51.29055667674642, 57.79700458310039), Array(49.096857069685576, 49.426658253716006, 49.83890973375404, 51.15811446987576, 57.589237558469144), Array(48.08721867597723, 48.37919401081916, 48.74416469398453, 49.91207088011423, 55.60561353749651), Array(48.55590657109014, 48.85703090352866, 49.23343631907681, 50.437933648830885, 56.30985813138201), Array(49.43616268229035, 49.76084171322781, 50.16669050189964, 51.46540662564948, 57.796647728929955), Array(49.248228839779685, 49.56675508123449, 49.964912883052996, 51.239017848872216, 57.45027955724091), Array(49.123821915465214, 49.42883356394992, 49.81009812455581, 51.03014471849464, 56.97787186394645), Array(49.43359733768894, 49.77477558345376, 50.16238042586881, 51.347555861470696, 57.13688723737948), Array(50.342975523728164, 50.640225071395434, 51.01178428663285, 52.20077377539075, 57.9970975330855), Array(50.8554216457819, 51.16549665040985, 51.553090406194784, 52.79339042470657, 58.839853014951544), Array(51.27769790851841, 51.605595511516164, 52.01546751526335, 53.32705792725436, 59.72106118571051), Array(50.66544190227365, 50.948789859082844, 51.30297480509434, 52.436366632331115, 57.96165179011041), Array(51.0476455006962, 51.3243

In [9]:
%scala
dfdd.write.csv("/FileStore/tables/predict2.csv")